In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import ee
import pandas as pd
import gee_utils

import math

In [44]:
# Trigger the authentication flow.
ee.Authenticate()


Successfully saved authorization token.


In [3]:
# Initialize the library.
ee.Initialize()

In [77]:
#Export parameters
EXPORT = 'drive'
BUCKET = None

# Directory setup
LSMS_EXPORT_FOLDER = 'lsms_tfrecords_raw'
LSMS_CSV_PATH = '../data/lsms_clusters.csv'

In [78]:
df_lsms_eth = pd.read_csv(LSMS_CSV_PATH)

df_lsms_eth

,country,consumption,density,lat,lon
0,ethiopia,7.114359,0,13.522697,39.440907
1,ethiopia,6.933497,1,9.602403,41.866592
2,ethiopia,7.354854,1,11.294477,36.448727
3,ethiopia,1.129382,1,9.503581,42.070496
4,ethiopia,1.924262,1,8.970294,36.525711
...,...,...,...,...,...
6508,ethiopia,4.283961,1,9.600062,41.245560
6509,ethiopia,2.158788,0,9.062462,38.757240
6510,ethiopia,1.986287,0,10.821666,40.187645
6511,ethiopia,2.943372,1,11.800598,37.128471


In [79]:
MS_BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP']

PROJECTION = 'EPSG:3857'
SCALE = 30              
EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
CHUNK_SIZE = 25         # set to a small number (<= 50) if Google Earth Engine reports memory errors

In [80]:
def export_images(df, country, export_folder, chunk_size):
    num_chunks = int(math.ceil(len(df) / chunk_size))
    tasks = {}

    for i in range(num_chunks):
        chunk_slice = slice(i * chunk_size, (i+1) * chunk_size - 1)
        fc = gee_utils.create_fc(df.loc[chunk_slice, :])

        roi = fc.geometry()
        composite_img = gee_utils.create_ms_img(roi).select(MS_BANDS)

        composite_img = gee_utils.add_latlon_band(composite_img)
        composite_img = gee_utils.add_nl_band(composite_img)
                
        fname = f'{country}_{i:02d}'
        tasks[(export_folder, country, i)] = gee_utils.make_tiles(
            img=composite_img, scale=SCALE, ksize=EXPORT_TILE_RADIUS,
            points=fc, export='drive',
            prefix=export_folder, fname=fname,
            bucket=None)
    return tasks

In [81]:
new_task = export_images(df_lsms_eth, country='ethiopia', export_folder=LSMS_EXPORT_FOLDER, chunk_size=CHUNK_SIZE)